# **Notebook 1: Data Extraction, Transformation and Loading**

## Objectives

* Import raw data from Kaggle into a dataframe
* Clean data to remove duplicate values and remove outliers
* Identify and handle missing data

## Inputs

* Raw data files from [CO2 Emissions Dataset](https://www.kaggle.com/datasets/shreyanshdangi/co-emissions-across-countries-regions-and-sectors/data)

## Outputs

* Generates clean_data.csv for use in hypothesis testing and visualisations

## Explanation of Terms

The interquartile range (IQR) was used to identify countries with high levels of cummulative or current CO2 emissions for inclusion in the analysis.<br>
IQR is a statistical measure of dispersion, with data being split into 4 quartiles. Q3 was used for our selection, meaning that we keep the top 75% of countries for each measure.



---

# Import Packages

Import packages needed to run the notebook

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt

# Import Data

Import raw data into dataframe, ready for processing

In [ ]:
# set path to data file
path = "../raw_data/data.csv"

# assign data to dataframe
df_raw = pd.read_csv(path)

# display dataframe
df_raw.head()

# Initital Data Cleaning

First I will do some initital data cleaning steps:
1. Check for duplicate rows and remove if found
2. Limit data to country information to fit the requirements of the analysis
3. Limit data to last 50 years to make analysis easier to manage and findings more relevant to current times

In [ ]:
# drop any duplicate rows
df_raw.drop_duplicates(inplace=True)

# select rows where Description is equal to Country
df_raw = df_raw.loc[df_raw['Description'] == "Country"]

# limit data to last 50 years
df_raw = df_raw.loc[df_raw["year"] >= 1975]

df_raw.shape

The next step is to drop columns which aren't needed for analysis, this will reduce the size of the data file making processing more efficient and the data easier to work with.<br>
(GPT-5 was used to format column names)

In [ ]:
# create a list of columns names to keep in the dataset
req_columns = ["Name", "iso_code", "year", "population", "gdp", "primary_energy_consumption", "co2", "co2_including_luc", "total_ghg", "co2_growth_abs", "co2_growth_prct",
               "co2_per_capita", "co2_per_gdp", "energy_per_capita", "energy_per_gdp", "cement_co2", "coal_co2", "flaring_co2", "gas_co2", "land_use_change_co2", "oil_co2",
               "share_global_co2", "share_global_co2_including_luc", "cumulative_co2", "cumulative_co2_including_luc", "share_global_cumulative_co2"]

# create a new dataframe with only the required columns
df_trimmed = df_raw[req_columns].copy()
df_trimmed.head()

The last step is to select countries with the highest cummulative or current co2 emission levels, so that analysis can focus on the most impactful countries for emissions.<br>
(Method and steps for data filtering suggested by copilot)

In [ ]:
# create a temporary dataframe with only the columns of interest for the most recent year
df_temp = df_trimmed.loc[df_trimmed['year'] == 2023, ['Name','cumulative_co2', 'co2']]

# calculate interquartile range, we are interested in the top 75% of values
iqr_cum, iqr_cap = df_temp[['cumulative_co2', 'co2']].quantile(.75)

# select countries where values are higher than IQR3
df_filtered = df_temp[(df_temp['cumulative_co2'] >= iqr_cum) | (df_temp['co2'] >= iqr_cap)]

# remove countries which do not meet the thresholds from the main dataset
df_trimmed = df_trimmed[df_trimmed['Name'].isin(df_filtered['Name'])].copy()
df_trimmed['Name'].value_counts().shape


---

# Initial Data Exploration

Next I will use y-data profiling to learn more about the dataset and find out if any further transformation is required to prepare it for use in visualisations.<br>
The preview from the table above shows some columns that will need to be rescaled (e.g. population and gdp) and some missing data to be handled (e.g. primary_energy_consumption and trade_co2_share).

In [ ]:
profile = ProfileReport(df=df_trimmed, minimal=True)
profile.to_notebook_iframe()

This has highlighted missing data in several columns which will need to be handled.<br>
Population data would benefit from being rescaled to a more human readable format. <br>
GDP is largly reported in scientific notation format and is not intuitive to interpret for most people, it could be beneficial to rescale this column. It is currently reported in $, but grouping as thousands or millions may make more sense.

---

# Basic Descriptives and Visualisations

This section includes basic data descriptives and visualisations which are used to inform final data transformations.<br>

## Population
Population data would benefit from being scaled to a more human readable format<br>
(GPT-5 was used to refactor the dataframe creation code and https://stackoverflow.com/questions/40347689/dataframe-describe-suppress-scientific-notation was used to reformat the descriptives output)

In [ ]:
# create dataframe with population data for 2023
df_temp = df_trimmed.loc[df_trimmed['year'] == 2023, ['Name','population']]
# reset dataframe index
df_temp.reset_index(drop=True)

# create descriptives for population and global co2 share, making sure format doesn't include scientific notation
df_temp.describe().apply(lambda s: s.apply('{0:.2f}'.format))

These values suggest that rescaling population into millions would be a good way to rescale the data<br>
(Code adapted from https://stackoverflow.com/questions/43675014/dividing-a-dataframe-column-and-then-rounding)

In [ ]:
# create a new population column by dividing population values by 1m and round to 2 decimal places
df_trimmed['pop(m)'] = df_trimmed['population'].div(1000000).round(2)
df_trimmed.head()

Here is a histogram to visualise population counts after rescaling

In [ ]:
# histogram of population
plt.hist(df_trimmed['pop(m)'])
plt.title("Histogram of Population Counts")
plt.xlabel("Population in millions")
plt.ylabel("Count")
plt.show()

This chart shows that most population data is on the lower end of the scale, even after removing low impact countries from analysis.<br>
There are a few outlier countries with very high populations, but as they are likely to also have higher emission levels these will remain.

## GDP
GDP data would benefit from being rescaled to a more human friendly format.

In [ ]:
df_trimmed['gdp'].describe()

Before rescaling I will need to handle missing values for GDP.<br>
(Code provided by GPT-5)

In [ ]:
# show countries with missing gdp in df_trimmed and how many missing rows per country
missing_counts = df_trimmed[df_trimmed['gdp'].isna()].groupby('Name').size().sort_values(ascending=False)
print(f"Total countries with missing gdp: {missing_counts.shape[0]}")
missing_counts.head()

All countries have some missing gdp data, but each country has at least some gdp data.<br>
I will use the fill up method to fill in missing values as it is assumed that earlier years are more likely to contain missing values than more recent ones. <br>
This method may skew earlier data as gdp is likely to increase over time.

In [ ]:
# upward fill missing GDP values
df_trimmed['gdp'].bfill(inplace=True)
# perform a forward fill to catch any remaining missing values
df_trimmed['gdp'].ffill(inplace=True)
df_trimmed['gdp'].info()


Now that all missing data has been filled we can rescale GDP.<br>
The minimum value is 142,968,000 (142 million) and the highest number is 26,966,000,000,000 (26 trillion).<br>
Rescaling GDP into billion dollars seems like a good method for producing values that can be understood intuitively.<br>
(https://www.calculatorsoup.com/calculators/math/scientific-notation-converter.php was used to convert scientific notation into real numbers)

In [ ]:
# create new gdp column by dividing gdp by 1b and rounding to 2 decimal places
df_trimmed['gdp($b)'] = df_trimmed['gdp'].div(1000000000).round(2)
df_trimmed.head()

Here's a histogram to show GDP after rescaling.

In [ ]:
# histogram of population
plt.hist(df_trimmed['gdp($b)'])
plt.title("Histogram of GDP Counts")
plt.xlabel("GDP in $billions")
plt.ylabel("Count")
plt.show()

Similarly to population most values are on the lower end of the scale, with a few higher outliers which we would expect to also contribute more highly to emissions.

## Missing Values
Now I need to identify and handle any remaining missing data.<br>
(Code improved by GPT-5)

In [ ]:
# show columns in df_trimmed that have missing values
missing = df_trimmed.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
missing

There are several columns with missing data, I want to identify any countries with no data for these metrics as these would not be good targets for back or forward fill methods.<br>
(Code written by GPT-5)

In [ ]:
# show countries with high missing values for a metric
missing_over_40 = []

for col in missing.index:
    counts = df_trimmed[df_trimmed[col].isna()].groupby('Name').size()
    for name, cnt in counts.items():
        if cnt > 40:
            missing_over_40.append((name, col, int(cnt)))

if missing_over_40:
    for name, col, cnt in missing_over_40:
        print(f"{name}: column '{col}' missing {cnt} rows")
else:
    print("No country has more than 40 missing values for any column")

My data owner (a copilot persona) has said that this missing data should not be imputed for accountability metrics, instead this missing data will be flagged in relevant visualisations.<br>
The rest of the missing values will be filled using the back/forward fill method using existing values, this may lead to some skew in time series analysis.<br>
(GPT-5 suggested code to guard against division errors)

In [ ]:
# fill missing energy values
df_trimmed['primary_energy_consumption'].bfill(inplace=True)
# perform a forward fill to catch any remaining missing values
df_trimmed['primary_energy_consumption'].ffill(inplace=True)

# gaps in calculated columns can now be filled using the imputed data
calc_col = ['energy_per_gdp', 'co2_per_gdp', 'energy_per_capita']

# calculate and fill missing values
df_trimmed['energy_per_gdp'] = df_trimmed['energy_per_gdp'].fillna(df_trimmed['primary_energy_consumption'] / df_trimmed['gdp'])
df_trimmed['co2_per_gdp'] = df_trimmed['co2_per_gdp'].fillna(df_trimmed['co2'] / df_trimmed['gdp'])
df_trimmed['energy_per_capita'] = df_trimmed['energy_per_capita'].fillna(df_trimmed['primary_energy_consumption'] / df_trimmed['population'])

# guard against division issues
df_trimmed.replace([np.inf, -np.inf], np.nan, inplace=True)

# show columns in df_trimmed that have missing values
missing = df_trimmed.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
missing

Now I need to handle the final missing data, these values will again be back/forward filled but North Korea and Taiwan will be handled separately.<br>
(Code generated by GPT-5)

In [ ]:
# define countries to exclude from imputation
exclude = ['Taiwan', 'North Korea']

# determine columns with missing values
cols_with_na = df_trimmed.columns[df_trimmed.isna().any()].tolist()

# perform bfill then ffill only for rows not in `exclude`
mask = ~df_trimmed['Name'].isin(exclude)
df_trimmed.loc[mask, cols_with_na] = df_trimmed.loc[mask, cols_with_na].bfill().ffill()

# replace any inf values that may have arisen, and show remaining missing counts
df_trimmed.replace([np.inf, -np.inf], np.nan, inplace=True)
df_trimmed[cols_with_na].isnull().sum().sort_values(ascending=False)


Finally I will fill in flaring CO2 values only for Taiwan

In [ ]:
# backfill flaring data, Taiwan is the only country with missing values, so we don't need any filtering
df_trimmed['flaring_co2'].bfill(inplace=True)
# forward fill in case any values were missed
df_trimmed['flaring_co2'].ffill(inplace=True)
df_trimmed.isnull().sum().sort_values(ascending=False)

The only remaining missing values are those flagged earlier as not to be imputed.<br>
As the last step in data transformation I will drop the raw values for population and GDP.

In [ ]:
# drop unnecessary columns
df_trimmed.drop(['gdp'], axis=1, inplace=True)
df_trimmed.drop(['population'], axis=1, inplace=True)

---

# Output Cleaned Data

Save cleaned data to a csv file ready to be used for visualisations, hypothesis testing, model building and dashboarding.

In [ ]:
# export cleaned data to csv
path = "../raw_data/clean_data.csv"
df_trimmed.to_csv(path, index=False)

This is the end of Notebook 1, users should open 02_statistics_and_visualisations.ipynb to continue analysis.